In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from datetime import datetime
import time
from concurrent.futures import ThreadPoolExecutor

In [2]:
def obtener_detalles_inmueble(session, url_inmueble):
    try:
        response = session.get(url_inmueble)
        response.raise_for_status()  # Verifica si la solicitud fue exitosa
        soup = BeautifulSoup(response.text, "html.parser")

        def get_text(selector, default="Error al acceder al dato"):
            try:
                element = soup.select_one(selector)
                return element.text.strip() if element else default
            except Exception as e:
                print(f"Error al extraer {selector}: {e}")
                return default

        try:
            nombre = get_text("h1.ad-title")
        except Exception as e:
            nombre = "Error al acceder al nombre"
            print(f"Error al extraer nombre: {e}")

        try:
            ubicacion = get_text("div.details__block > p", "Error al acceder a la ubicación")
        except Exception as e:
            ubicacion = "Error al acceder a la ubicación"
            print(f"Error al extraer ubicación: {e}")

        try:
            agencia = get_text("p.owner-info__name a", "Agencia no disponible")
        except Exception as e:
            agencia = "Agencia no disponible"
            print(f"Error al extraer agencia: {e}")

        try:
            precio_m2 = next((item.text.strip() for item in soup.select("ul.features-summary li.features-summary__item") if "€/m²" in item.text), "Error al procesar precio m²")
        except Exception as e:
            precio_m2 = "Error al procesar precio m²"
            print(f"Error al extraer precio m²: {e}")

        try:
            precio_element = soup.select_one("div.price__value.jsPriceValue")
            if precio_element:
                precio = precio_element.text.strip()
                unidad_tiempo_element = soup.select_one("select.price__selector.jsPriceSelector option[selected]")
                if unidad_tiempo_element:
                    unidad_tiempo = unidad_tiempo_element.text.strip()
                    precio_completo = f"{precio} / {unidad_tiempo}"
                else:
                    precio_completo = precio
            else:
                precio_completo = "Error al procesar precio"
        except Exception as e:
            precio_completo = "Error al procesar precio completo"
            print(f"Error al extraer precio: {e}")

        try:
            superficie = get_text("span.features__value", "Error al procesar superficie")
        except Exception as e:
            superficie = "Error al procesar superficie"
            print(f"Error al extraer superficie: {e}")

        try:
            actualizacion = get_text("div.details__block.last-update", "Error al procesar actualización").replace("Última actualización\n", "").strip()
        except Exception as e:
            actualizacion = "Error al procesar actualización"
            print(f"Error al extraer actualización: {e}")

        try:
            certificado_energetico_etiquetas = [etiqueta['class'][1].split('--')[1].strip().upper() for etiqueta in soup.select("span.energy-certificate__tag") if 'energy-certificate__tag--' in etiqueta['class'][1]]
            certificado_energetico = ', '.join(certificado_energetico_etiquetas) if certificado_energetico_etiquetas else "En trámite"
        except Exception as e:
            certificado_energetico = "Error al procesar certificado energético"
            print(f"Error al extraer certificado energético: {e}")

        try:
            detalles = {feature.select_one("span.features__label").text.strip().replace(":", ""): feature.select_one("span.features__value").text.strip() if feature.select_one("span.features__value") else "Si" for feature in soup.select("div.features__feature")}
        except Exception as e:
            detalles = {}
            print(f"Error al extraer detalles: {e}")

        try:
            identificador = re.search(r'-(\d+_\d+)', url_inmueble).group(1) if re.search(r'-(\d+_\d+)', url_inmueble) else 'NaN'
        except Exception as e:
            identificador = 'NaN'
            print(f"Error al extraer identificador: {e}")

        timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

        try:
            location_div = soup.select_one("div.location")
            data_params = location_div["data-params"] if location_div else ""
            latitud = re.search(r'latitude=([-0-9.]+)', data_params).group(1) if re.search(r'latitude=([-0-9.]+)', data_params) else 'NaN'
            longitud = re.search(r'longitude=([-0-9.]+)', data_params).group(1) if re.search(r'longitude=([-0-9.]+)', data_params) else 'NaN'
            coordenadas = f"{latitud}, {longitud}"
        except Exception as e:
            coordenadas = "NaN, NaN"
            print(f"Error al extraer coordenadas: {e}")

        return nombre, ubicacion, agencia, precio_m2, precio_completo, superficie, actualizacion, certificado_energetico, detalles, identificador, timestamp, coordenadas
    except Exception as e:
        print(f"Error al procesar el inmueble {url_inmueble}: {e}")
        return ("Error al acceder al inmueble",) * 8 + ({}, "", "", datetime.now().strftime('%Y-%m-%d %H:%M:%S'), "")

In [3]:
def obtener_inmuebles_de_urls(urls_a_scrapear):
    todos_los_inmuebles = []
    columnas = set()
    identificadores_unicos = set()
    session = requests.Session()  # Usar una sesión para mantener conexiones persistentes

    def procesar_url(url_pagina):
        try:
            print(f"Procesando la URL: {url_pagina}")
            response = session.get(url_pagina)
            if response.status_code == 404:
                print(f"Página no encontrada: {url_pagina}")
                return
            
            soup = BeautifulSoup(response.text, "lxml")
            titulos_soup = soup.find_all("a", class_="ad-preview__title")
            
            for titulo_soup in titulos_soup:
                nombre = titulo_soup.text.strip()
                href = "https://www.pisos.com" + titulo_soup['href']
                print(f"Procesando inmueble: {href}")
                nombre_detalle, ubicacion, agencia, precio_m2, precio, superficie, actualizacion, certificado_energetico, detalles, identificador, timestamp, coordenadas = obtener_detalles_inmueble(session, href)
                
                if identificador in identificadores_unicos:
                    print(f"Inmueble duplicado: {identificador}")
                    continue  # Saltar si el identificador ya existe
                
                identificadores_unicos.add(identificador)
                
                inmueble = {
                    "nombre": nombre,
                    "ubicacion": ubicacion,
                    "agencia": agencia,
                    "precio_m2": precio_m2,
                    "precio": precio,
                    "superficie": superficie,
                    "href": href,
                    "actualizacion": actualizacion,
                    "certificado_energetico": certificado_energetico,
                    "identificador": identificador,
                    "timestamp": timestamp,
                    "coordenadas": coordenadas
                }
                
                inmueble.update(detalles)
                todos_los_inmuebles.append(inmueble)
                columnas.update(inmueble.keys())
        
        except Exception as e:
            print(f"Error al procesar la URL {url_pagina}: {e}")

    with ThreadPoolExecutor(max_workers=10) as executor:
        executor.map(procesar_url, urls_a_scrapear)
    
    print(f"Total de inmuebles procesados: {len(todos_los_inmuebles)}")
    df_inmuebles = pd.DataFrame(todos_los_inmuebles, columns=list(columnas))
    return df_inmuebles

In [4]:
# Leer el archivo txt con las URLs y asignarlas a la lista 'urls'
file_path = 'urls_scrap_alquileres.txt'
try:
    with open(file_path, 'r') as file:
        urls = file.read().splitlines()
    print(f"Se han cargado {len(urls)} URLs desde el archivo.")
except FileNotFoundError:
    print(f"El archivo {file_path} no se encontró.")

Se han cargado 56 URLs desde el archivo.


In [5]:
# Función para verificar si una página existe
def pagina_existe(session, url):
    response = session.get(url)
    if response.status_code != 200:
        return False
    soup = BeautifulSoup(response.text, 'html.parser')
    no_results = soup.find('div', class_='no-results')
    if no_results:
        return False
    return True

# Generar lista con las páginas de cada una de las URLs a scrapear
def generar_urls_a_scrapear(urls):
    urls_a_scrapear = []
    session = requests.Session()  # Usar una sesión para mantener conexiones persistentes

    for base_url in urls:  # Intentar si hay hasta la página 100
        for i in range(1, 101):
            url = f"{base_url}{i}/"
            if pagina_existe(session, url):
                urls_a_scrapear.append(url)
                print(f"Página encontrada: {url}")
            else:
                print(f"Página no encontrada: {url}")
                break  # Si no existe la página, lo dejo
    print(f"Hay {len(urls_a_scrapear)} URLs a scrapear")
    return urls_a_scrapear
urls_a_scrapear = generar_urls_a_scrapear(urls)

Página encontrada: https://www.pisos.com/alquiler/pisos-madrid/1/
Página encontrada: https://www.pisos.com/alquiler/pisos-madrid/2/
Página encontrada: https://www.pisos.com/alquiler/pisos-madrid/3/
Página encontrada: https://www.pisos.com/alquiler/pisos-madrid/4/
Página encontrada: https://www.pisos.com/alquiler/pisos-madrid/5/
Página encontrada: https://www.pisos.com/alquiler/pisos-madrid/6/
Página encontrada: https://www.pisos.com/alquiler/pisos-madrid/7/
Página encontrada: https://www.pisos.com/alquiler/pisos-madrid/8/
Página encontrada: https://www.pisos.com/alquiler/pisos-madrid/9/
Página encontrada: https://www.pisos.com/alquiler/pisos-madrid/10/
Página encontrada: https://www.pisos.com/alquiler/pisos-madrid/11/
Página encontrada: https://www.pisos.com/alquiler/pisos-madrid/12/
Página encontrada: https://www.pisos.com/alquiler/pisos-madrid/13/
Página encontrada: https://www.pisos.com/alquiler/pisos-madrid/14/
Página encontrada: https://www.pisos.com/alquiler/pisos-madrid/15/
Pági

In [6]:
# Obtener inmuebles de las URLs generadas
df_inmuebles = obtener_inmuebles_de_urls(urls_a_scrapear)

Procesando la URL: https://www.pisos.com/alquiler/pisos-madrid/1/
Procesando la URL: https://www.pisos.com/alquiler/pisos-madrid/2/
Procesando la URL: https://www.pisos.com/alquiler/pisos-madrid/3/
Procesando la URL: https://www.pisos.com/alquiler/pisos-madrid/4/
Procesando la URL: https://www.pisos.com/alquiler/pisos-madrid/5/
Procesando la URL: https://www.pisos.com/alquiler/pisos-madrid/6/
Procesando la URL: https://www.pisos.com/alquiler/pisos-madrid/7/
Procesando la URL: https://www.pisos.com/alquiler/pisos-madrid/8/
Procesando la URL: https://www.pisos.com/alquiler/pisos-madrid/9/
Procesando la URL: https://www.pisos.com/alquiler/pisos-madrid/10/
Procesando inmueble: https://www.pisos.com/alquilar/piso-tetuan_cuatro_caminos28020-50932627354_108900/
Procesando inmueble: https://www.pisos.com/alquilar/atico-justicia_chueca28004-945856909480201_109700/
Procesando inmueble: https://www.pisos.com/alquilar/estudio-pilar28029-49208311379_100500/
Procesando inmueble: https://www.pisos.co

In [7]:
df_inmuebles.shape

(14635, 66)

In [8]:
df_inmuebles.columns

Index(['Calefacción', 'Soleado', 'superficie', 'No se aceptan mascotas',
       'Zona de juegos infantiles', 'Urbanizado', 'timestamp',
       'Calle asfaltada', 'Cocina equipada', 'Superficie construida',
       'Superficie solar', 'Jardín', 'Habitaciones', 'Luz', 'Zona comunitaria',
       'Superficie útil', 'Carpintería exterior', 'ubicacion', 'Tipo de casa',
       'Referencia', 'Trastero', 'actualizacion', 'Agua',
       'Sistema de seguridad', 'Chimenea', 'precio_m2', 'Teléfono', 'Garaje',
       'Tipo suelo', 'Instalaciones deportivas', 'agencia', 'Alcantarillado',
       'Aire acondicionado', 'Vidrios dobles', 'Terraza', 'Aerotermia',
       'identificador', 'Puerta blindada', 'Lavadero', 'nombre', 'Comedor',
       'Gas', 'Ascensor', 'Se aceptan mascotas', 'certificado_energetico',
       'Gastos de comunidad', 'Adaptado a personas con movilidad reducida',
       'Orientación', 'Cuarto de bicicletas', 'Interior', 'Conservación',
       'Planta', 'precio', 'Baños', 'Armarios em

In [9]:
df_inmuebles

,Calefacción,Soleado,superficie,No se aceptan mascotas,Zona de juegos infantiles,Urbanizado,timestamp,Calle asfaltada,Cocina equipada,Superficie construida,...,Portero automático,Balcón,Piscina,Antigüedad,Calle alumbrada,href,Exterior,Amueblado,Sala comunitaria,Carpintería interior
0,Gas natural,Si,350 m²,NaN,NaN,NaN,2025-03-06 15:26:29,NaN,Si,350 m²,...,Si,Si,NaN,Más de 50 años,NaN,https://www.pisos.com/alquilar/atico-justicia_...,NaN,NaN,NaN,NaN
1,Gas natural,NaN,85 m²,NaN,NaN,NaN,2025-03-06 15:26:29,NaN,NaN,85 m²,...,NaN,Si,NaN,Más de 50 años,NaN,https://www.pisos.com/alquilar/piso-trafalgar2...,NaN,NaN,NaN,NaN
2,Si,NaN,100 m²,NaN,NaN,NaN,2025-03-06 15:26:29,NaN,Si,100 m²,...,NaN,NaN,Si,NaN,NaN,https://www.pisos.com/alquilar/piso-tetuan_cua...,Si,NaN,NaN,NaN
3,NaN,NaN,47 m²,NaN,NaN,NaN,2025-03-06 15:26:29,NaN,Cocina amueblada,47 m²,...,NaN,NaN,NaN,Más de 50 años,NaN,https://www.pisos.com/alquilar/estudio-pilar28...,Si,NaN,NaN,NaN
4,Eléctrica,NaN,73 m²,NaN,NaN,NaN,2025-03-06 15:26:29,NaN,Si,73 m²,...,Si,NaN,NaN,Más de 50 años,NaN,https://www.pisos.com/alquilar/apartamento-sol...,Si,A estrenar,NaN,Si
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14630,Eléctrica,NaN,255 m²,Si,NaN,NaN,2025-03-06 15:52:05,NaN,Abierta a comedor,255 m²,...,NaN,NaN,NaN,NaN,NaN,https://www.pisos.com/alquilar/chalet-angoustr...,NaN,Si,NaN,Madera
14631,NaN,NaN,70 m²,Si,NaN,NaN,2025-03-06 15:52:06,NaN,NaN,70 m²,...,NaN,NaN,NaN,Entre 5 y 10 años,NaN,https://www.pisos.com/alquilar/apartamento-sai...,Si,Si,NaN,NaN
14632,Gasoil,Si,55 m²,NaN,NaN,NaN,2025-03-06 15:52:06,NaN,Abierta,55 m²,...,NaN,NaN,NaN,Entre 5 y 10 años,NaN,https://www.pisos.com/alquilar/apartamento-err...,Si,Si,NaN,Madera
14633,Eléctrica,Orientacion este,36 m²,Si,NaN,NaN,2025-03-06 15:52:07,NaN,Abierta,36 m²,...,Si,NaN,Comunitaria,NaN,NaN,https://www.pisos.com/alquilar/apartamento-bou...,Con jardin privado,Si,NaN,Si


In [10]:
# Función para comprobar si "Error" está en alguna celda y devolver las ubicaciones
def contiene_error(df):
    error_locations = df.apply(lambda col: col.astype(str).str.contains('Error'))
    if error_locations.any().any():
        return error_locations
    else:
        return None

# Llamada a la función
error_locations = contiene_error(df_inmuebles)
if error_locations is not None:
    print("Hay al menos un valor con la palabra 'Error' en el DataFrame en las siguientes ubicaciones:")
    # Filtrar solo las filas y columnas con errores
    error_indices = error_locations[error_locations].stack().index
    for row, col in error_indices:
        print(f"Fila: {row}, Columna: {col}")
else:
    print("No hay valores con la palabra 'Error' en el DataFrame.")

Hay al menos un valor con la palabra 'Error' en el DataFrame en las siguientes ubicaciones:
Fila: 123, Columna: precio_m2
Fila: 213, Columna: precio_m2
Fila: 375, Columna: precio_m2
Fila: 732, Columna: precio_m2
Fila: 1134, Columna: precio_m2
Fila: 1243, Columna: precio_m2
Fila: 1324, Columna: precio_m2
Fila: 1646, Columna: precio_m2
Fila: 1830, Columna: precio_m2
Fila: 1859, Columna: precio_m2
Fila: 2368, Columna: precio_m2
Fila: 2373, Columna: precio_m2
Fila: 2382, Columna: precio_m2
Fila: 2407, Columna: precio_m2
Fila: 2429, Columna: precio_m2
Fila: 2476, Columna: precio_m2
Fila: 2489, Columna: precio_m2
Fila: 2493, Columna: precio_m2
Fila: 2504, Columna: precio_m2
Fila: 2525, Columna: precio_m2
Fila: 2609, Columna: precio_m2
Fila: 2613, Columna: precio_m2
Fila: 2659, Columna: precio_m2
Fila: 2727, Columna: precio_m2
Fila: 2738, Columna: precio_m2
Fila: 2748, Columna: precio_m2
Fila: 2752, Columna: precio_m2
Fila: 2762, Columna: precio_m2
Fila: 2778, Columna: precio_m2
Fila: 2780, C

In [11]:
df_seleccionado = df_inmuebles[["identificador", "nombre", "ubicacion", "actualizacion", "precio", "timestamp", "coordenadas", "href"]]

In [12]:
df_seleccionado

,identificador,nombre,ubicacion,actualizacion,precio,timestamp,coordenadas,href
0,945856909480201_109700,Ático en calle de Sagasta,Justicia-Chueca (Distrito Centro. Madrid Capital),Anuncio actualizado el 28/02/2025,8.500 €/mes,2025-03-06 15:26:29,"40.427678, -3.6966561",https://www.pisos.com/alquilar/atico-justicia_...
1,51704869757_100500,Piso en calle de Santa Feliciana,Trafalgar (Distrito Chamberí. Madrid Capital),Anuncio actualizado el 27/02/2025,3.000 €/mes,2025-03-06 15:26:29,"40.4333435, -3.6992494",https://www.pisos.com/alquilar/piso-trafalgar2...
2,50932627354_108900,Piso en Tetuan,Cuatro Caminos (Distrito Tetuán. Madrid Capital),Anuncio actualizado el 06/03/2025,1.900 €/mes,2025-03-06 15:26:29,"40.4590003, -3.6985013",https://www.pisos.com/alquilar/piso-tetuan_cua...
3,49208311379_100500,Estudio en calle de Ginzo de Limia,Pilar (Distrito Fuencarral-El Pardo. Madrid Ca...,Anuncio actualizado el 30/01/2025,1.100 €/mes,2025-03-06 15:26:29,"40.477914514, -3.703535662",https://www.pisos.com/alquilar/estudio-pilar28...
4,50847096530_528950,"Apartamento en calle del Arenal, 8",Sol (Distrito Centro. Madrid Capital),Anuncio actualizado el 11/02/2025,1.600 €/mes,2025-03-06 15:26:29,"40.417088, -3.7055147",https://www.pisos.com/alquilar/apartamento-sol...
...,...,...,...,...,...,...,...,...
14630,39243632341_517227,"Chalet en calle Major, nº 1",Angoustrine-Villeneuve-des-Escaldes,Anuncio actualizado el 12/11/2024,1.825 €/mes,2025-03-06 15:52:05,"42.484920537025346, 1.9410527164669382",https://www.pisos.com/alquilar/chalet-angoustr...
14631,50887588369_517227,Apartamento en Carrer de Francesc Macià,Sainte-Léocadie,Anuncio actualizado el 14/01/2025,900 €/mes,2025-03-06 15:52:06,"42.369853264, 1.775765794",https://www.pisos.com/alquilar/apartamento-sai...
14632,49206973095_517227,"Apartamento en calle Canigo, nº 3",Err,Anuncio actualizado el 08/12/2024,650 €/mes,2025-03-06 15:52:06,"42.43690985362229, 2.0173286672415274",https://www.pisos.com/alquilar/apartamento-err...
14633,36693113219_517227,Apartamento en calle Caldegas,Bourg-Madame,Anuncio actualizado el 01/03/2025,1.300 €/mes,2025-03-06 15:52:07,"42.4338622055752, 1.944115971585786",https://www.pisos.com/alquilar/apartamento-bou...


In [13]:
df_seleccionado["precio"].unique().tolist()

['8.500 €/mes',
 '3.000 €/mes',
 '1.900 €/mes',
 '1.100 €/mes',
 '1.600 €/mes',
 '1.400 €/mes',
 '3.200 €/mes',
 '4.000 €/mes',
 '1.200 €/mes',
 '1.236 €/mes',
 '3.450 €/mes',
 '1.750 €/mes',
 '3.500 €/mes',
 '1.050 €/mes',
 '1.500 €/mes',
 '2.300 €/mes',
 '3.600 €/mes',
 '3.900 €/mes',
 '1.195 €/mes',
 '1.000 €/mes',
 '1.840 €/mes',
 '4.950 €/mes',
 '840 €/mes',
 '7.500 €/mes',
 '2.500 €/mes',
 '2.250 €/mes',
 '980 €/mes',
 '1.500 € / mes',
 '1.520 €/mes',
 '4.250 €/mes',
 '1.800 €/mes',
 '1.810 €/mes',
 '4.800 €/mes',
 '1.298 €/mes',
 '4.100 €/mes',
 '875 €/mes',
 '1.350 €/mes',
 '2.000 €/mes',
 '1.450 €/mes',
 '700 €/mes',
 '3.700 € / mes',
 '1.505 €/mes',
 '1.150 €/mes',
 '890 €/mes',
 '1.250 €/mes',
 '950 €/mes',
 '975 €/mes',
 '1.550 €/mes',
 '1.147 €/mes',
 '1.990 €/mes',
 '850 €/mes',
 '1.765 €/mes',
 '3.735 €/mes',
 '1.695 €/mes',
 '2.050 €/mes',
 '1.545 €/mes',
 '1.845 €/mes',
 '2.800 €/mes',
 '1.089 €/mes',
 '1.575 €/mes',
 '1.770 €/mes',
 '4.300 €/mes',
 '1.700 €/mes',
 '1.

In [14]:
df_inmuebles.to_csv('alquileres_completo.csv', index=False)